# Deploying Personlab pose prediction model

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

In [ ]:
from azureml.core.model import Model

### Registering the model to the workspace

In [ ]:
#skip if model is already registered
model = Model.register(model_path = "./personlab/",
                       model_name = "personlabV1",
                       description = "personalab model for detecting poses",
                       workspace = ws)

### If the model is already registered in the workspace fetch it using the model name

In [ ]:
model = Model(ws, name='personlabV1')

In [ ]:
model

In [ ]:
#skip if model is already downloaded
model.download(target_dir='./personlab', exist_ok=False, exists_ok=None)

### Entry script for deploying on a AKS(Azure Kubernetes Service)

In [ ]:
%%writefile score_aks.py
import json
import numpy as np
import os
import sys
import time
from scipy.ndimage.filters import gaussian_filter
import cv2
import tensorflow as tf
import time

sys.path.append('/var/azureml-app/azureml-models/personlabV1/2/personlab')
import utils

#tf.compat.v1.disable_eager_execution()

def init():
    print(time.time())
    MODEL_DIR = '/var/azureml-app/azureml-models/personlabV1/2/personlab/'
    multiscale = [1.0,1.5,2.0]
        
    global tf_img
    tf_img = []
    global outputs 
    outputs = []
    for i in range(len(multiscale)):
        scale = multiscale[i]
        tf_img.append(tf.placeholder(tf.float32,shape=[1,int(scale*401),int(scale*401),3]))
        outputs.append(utils.model(tf_img[i])) 
    global sess
    sess= tf.Session()

    global_vars = tf.global_variables()
    saver = tf.train.Saver(var_list = global_vars)
    checkpoint_path = MODEL_DIR+'model.ckpt'
    saver.restore(sess,checkpoint_path)
    print(time.time())

def run(data):
    
    try:
        print(time.time())
       #TODO find logger in 
        multiscale = [1.0,1.5,2.0]
        batch_size,height,width=1,401,401
        image_list = json.loads(data)
        pose_scoreslist=[]
        pose_keypoint_scoreslist=[]
        pose_keypoint_coordslist=[]
        
        
        for i in range(1):
            if(i==0):
                input_image = np.array(image_list['input_image1'], dtype=np.uint8)
            else:
                input_image = np.array(image_list['input_image2'], dtype=np.uint8)

            scale_outputs = []
            for i in range(len(multiscale)):
                scale = multiscale[i]
                cv_shape = (401, 401)
                cv_shape2 = (int(cv_shape[0]*scale),int(cv_shape[1]*scale))
                scale2=cv_shape2[0]/600
                input_img = cv2.resize(input_image,None,fx=scale2,fy=scale2)
                #input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB).astype(np.float32)
                input_img = cv2.copyMakeBorder(input_img,0,cv_shape2[0]-input_img.shape[0],0,cv_shape2[1]-input_img.shape[1],cv2.BORDER_CONSTANT,value=[127,127,127])
                scale_img =input_img
                imgs_batch = np.zeros((batch_size,int(scale*height),int(scale*width),3))
                imgs_batch[0] = scale_img

                one_scale_output = sess.run(outputs[i],feed_dict={tf_img[i]:imgs_batch})
                scale_outputs.append([o[0] for o in one_scale_output])

            sample_output = scale_outputs[0]
            for i in range(1,len(multiscale)):
                for j in range(len(sample_output)):
                    sample_output[j]+=scale_outputs[i][j]
            for j in range(len(sample_output)):
                sample_output[j] /=len(multiscale)

            H = utils.compute_heatmaps(kp_maps=sample_output[0], short_offsets=sample_output[1])
            for i in range(17):
                H[:,:,i] = gaussian_filter(H[:,:,i], sigma=2)

            pred_kp = utils.get_keypoints(H)
            pred_skels = utils.group_skeletons(keypoints=pred_kp, mid_offsets=sample_output[2])
            pred_skels = [skel for skel in pred_skels if (skel[:,2]>0).sum() > 6]
            #print ('Number of detected skeletons: {}'.format(len(pred_skels)))

            pose_scores = np.zeros(len(pred_skels))
            pose_keypoint_scores = np.zeros((len(pred_skels), 17))
            pose_keypoint_coords = np.zeros((len(pred_skels), 17, 2))

            for j in range(len(pred_skels)):
                sum=0;
                for i in range(17):
                    sum+=pred_skels[j][i][2]*100
                    pose_keypoint_scores[j][i]=pred_skels[j][i][2]*100
                    pose_keypoint_coords[j][i][0]=pred_skels[j][i][0]
                    pose_keypoint_coords[j][i][1]=pred_skels[j][i][1]
                pose_scores[j]=sum/17
                
            pose_scoreslist.append(pose_scores)
            pose_keypoint_scoreslist.append(pose_keypoint_scores)
            pose_keypoint_coordslist.append(pose_keypoint_coords)
            

        result = json.dumps({'pose_scores': pose_scoreslist, 'keypoint_scores': pose_keypoint_scoreslist,'keypoint_coords':pose_keypoint_coordslist})
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error

### Entry script to deploy to ACI(Azure Container Instance)

In [ ]:
%%writefile score_aci.py
import json
import numpy as np
import os
import sys
import time
from scipy.ndimage.filters import gaussian_filter
import cv2
import tensorflow as tf

sys.path.append('/var/azureml-app/azureml-models/personlabV1/2/personlab')
import utils

#tf.compat.v1.disable_eager_execution()

def init(): 
    MODEL_DIR = '/var/azureml-app/azureml-models/personlabV1/2/personlab/'
    multiscale = [1.0,1.5,2.0]
        
    global tf_img
    tf_img = []
    global outputs 
    outputs = []
    for i in range(len(multiscale)):
        scale = multiscale[i]
        tf_img.append(tf.placeholder(tf.float32,shape=[1,int(scale*401),int(scale*401),3]))
        outputs.append(utils.model(tf_img[i])) 
    global sess
    sess= tf.Session()

    global_vars = tf.global_variables()
    saver = tf.train.Saver(var_list = global_vars)
    checkpoint_path = MODEL_DIR+'model.ckpt'
    saver.restore(sess,checkpoint_path)

def run(data):
    
    try:
       #TODO find logger in 
        multiscale = [1.0,1.5,2.0]
        batch_size,height,width=1,401,401
        image_list = json.loads(data)
        input_image = np.array(image_list['input_image'], dtype=np.uint8)
         
        scale_outputs = []
        for i in range(len(multiscale)):
            scale = multiscale[i]
            cv_shape = (401, 401)
            cv_shape2 = (int(cv_shape[0]*scale),int(cv_shape[1]*scale))
            scale2=cv_shape2[0]/600
            input_img = cv2.resize(input_image,None,fx=scale2,fy=scale2)
            #input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB).astype(np.float32)
            input_img = cv2.copyMakeBorder(input_img,0,cv_shape2[0]-input_img.shape[0],0,cv_shape2[1]-input_img.shape[1],cv2.BORDER_CONSTANT,value=[127,127,127])
            scale_img =input_img
            imgs_batch = np.zeros((batch_size,int(scale*height),int(scale*width),3))
            imgs_batch[0] = scale_img

            one_scale_output = sess.run(outputs[i],feed_dict={tf_img[i]:imgs_batch})
            scale_outputs.append([o[0] for o in one_scale_output])
            
        sample_output = scale_outputs[0]
        for i in range(1,len(multiscale)):
            for j in range(len(sample_output)):
                sample_output[j]+=scale_outputs[i][j]
        for j in range(len(sample_output)):
            sample_output[j] /=len(multiscale)
            
        H = utils.compute_heatmaps(kp_maps=sample_output[0], short_offsets=sample_output[1])
        for i in range(17):
            H[:,:,i] = gaussian_filter(H[:,:,i], sigma=2)
        
        pred_kp = utils.get_keypoints(H)
        pred_skels = utils.group_skeletons(keypoints=pred_kp, mid_offsets=sample_output[2])
        pred_skels = [skel for skel in pred_skels if (skel[:,2]>0).sum() > 6]
        #print ('Number of detected skeletons: {}'.format(len(pred_skels)))
        
        pose_scores = np.zeros(len(pred_skels))
        pose_keypoint_scores = np.zeros((len(pred_skels), 17))
        pose_keypoint_coords = np.zeros((len(pred_skels), 17, 2))

        for j in range(len(pred_skels)):
            sum=0;
            for i in range(17):
                sum+=pred_skels[j][i][2]*100
                pose_keypoint_scores[j][i]=pred_skels[j][i][2]*100
                pose_keypoint_coords[j][i][0]=pred_skels[j][i][0]
                pose_keypoint_coords[j][i][1]=pred_skels[j][i][1]
            pose_scores[j]=sum/17
            
        result = json.dumps({'pose_scores': pose_scores.tolist(), 'keypoint_scores': pose_keypoint_scores.tolist(),'keypoint_coords':pose_keypoint_coords.tolist()})
       
        # You can return any data type, as long as it is JSON serializable.
        return result
    except Exception as e:
        error = str(e)
        return error

### Define the dependencies required for the model and set inference config

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment('personlab')
env.python.conda_dependencies = CondaDependencies.create(pip_packages=['tensorflow==1.13.2','matplotlib==3.1.1','azureml-defaults', 'scipy==1.3.1', 'inference_schema', 'opencv-python==3.4.5.20', 'scikit-image==0.15.0','scikit-learn==0.21.3', 'psycopg2-binary', 'tqdm', 'Pillow'])
inference_config_aci = InferenceConfig(entry_script="score_aci.py", environment=env)
inference_config_aks = InferenceConfig(entry_script="score_aks.py", environment=env)

### Use to register the environment to the workspace

In [ ]:
env.register(ws)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

### Specify the deployment config and deploy the service

In [ ]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 2, memory_gb = 16,location = "centralindia")
service = Model.deploy(ws, "aci-personlabv1", [model], inference_config_aci, deployment_config)
service.wait_for_deployment(show_output = True)
print(service.state)

### check the service logs if things don't work

In [ ]:
service.get_logs()

## Creating Inference cluster

For provisioning_configuration(), if you pick custom values for agent_count and vm_size, and cluster_purpose is not DEV_TEST, then you need to make sure agent_count multiplied by vm_size is greater than or equal to 12 virtual CPUs. For example, if you use a vm_size of "Standard_D3_v2", which has 4 virtual CPUs, then you should pick an agent_count of 3 or greater.

### skip if already created

In [ ]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this).
# For example, to create a dev/test cluster, use:
# prov_config = AksCompute.provisioning_configuration(cluster_purpose = AksCompute.ClusterPurpose.DEV_TEST)
prov_config = AksCompute.provisioning_configuration()

aks_name = 'webservices'

prov_config = prov_config = AksCompute.provisioning_configuration(vm_size = "Standard_A2_v2",
                                                       agent_count = 6,
                                                       location = "centralindia")

# Create the cluster
aks_target = ComputeTarget.create(workspace = ws,
                                    name = aks_name,
                                    provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)

In [ ]:
from azureml.core.webservice import AksWebservice, Webservice
from azureml.core.compute import AksCompute

### Deploy the model on created AKS

In [ ]:
aks_target = AksCompute(ws,"webservices")

deployment_config = AksWebservice.deploy_configuration(cpu_cores = 6, memory_gb = 64,autoscale_enabled=True, autoscale_max_replicas=3, collect_model_data=True, enable_app_insights=True)
service = Model.deploy(ws, "aks-personlab-eur-test1", [model], inference_config_aks, deployment_config,aks_target)
service.wait_for_deployment(show_output = True)
print(service.state)

In [ ]:
service.get_logs()